## Preparation

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)

import os
import glob

## data

In [2]:
data_dir = '../input/'

In [3]:
test = pd.read_csv('../input/test.csv')
train = pd.read_csv('../input/train.csv')

train["row_id"] = train["stock_id"].astype(str)+"-"+train["time_id"].astype(str)
test["row_id"] = test["stock_id"].astype(str)+"-"+test["time_id"].astype(str)

### book_train.parquet  
オーダーbookデータは予約状況  
この値段まで下がってきたら、買う  
上がってきたら売るみたいな予約情報

### trade_train.parque
リアルタイムで実際に取引された量  
買い手がこの値段でこの数量買い、売り手がこの値段でこの数量売ったなど  


stock_id : 株の銘柄（どの株か）  
time_id : どの時間の情報かのid (submissionファイルのtime_idと連動しています)  
seconds_in_bucket : time_idの中で、0からスタートして何秒後か。たぶん予測するのは、10分のtotalなので、seconds_in_bucketは、最大600 secのはず  
bid_price1,2 : 株の買値の希望値の１番目と２番目 ※　(Normalized prices of the most/second most competitive buy level. だから、正確には、１番と２番目に正規化されたレベルの買値。
→買値の希望値をみんな出しているけど、それの正規化したときに一番多い値と２番目に多い値と推測。以下askも逆の現象。)  

ask_price1,2 : 株の売り値の希望値  
bid_size1,2 : 買うのを希望している側の１番目と２番目の株式数  
ask_size1,2 : 売るのを希望している側の１番目と２番目の株式数  

In [4]:
# book_example = pd.read_parquet('../input/book_train.parquet/stock_id=1')
# book_example


## Functions for preprocess

In [5]:
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

In [6]:
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

In [7]:
def log_return(series):
    return np.log(series).diff()

# 1行目はNullになるので注意

In [8]:
def realized_vocatility(series):
    return np.sqrt(np.sum(series**2))
    

In [9]:
def count_unique(series):
    return len(np.unique(series))

## Feature Engineering

In [10]:
stock = train.groupby("stock_id")['target'].agg(["mean","median","std","count","sum"]).reset_index()
time = train.groupby(["time_id"])["target"].agg(["mean","median","std","count","sum"]).reset_index()
# row = train.groupby(["row_id"])["target"].agg(["mean","median","std","count","sum"]).reset_index()

In [11]:

train_info = train.copy()
train_info['stock_id_mean']     = train['stock_id'].map(dict(zip(stock['stock_id'], stock['mean'])))
train_info['stock_id_median']   = train['stock_id'].map(dict(zip(stock['stock_id'], stock['median'])))
train_info['stock_id_std']      = train['stock_id'].map(dict(zip(stock['stock_id'], stock['std'])))

train_info['time_id_mean']      = train['time_id'].map(dict(zip(time['time_id'], time['mean'])))
train_info['time_id_median']    = train['time_id'].map(dict(zip(time['time_id'], time['median'])))
train_info['time_id_std']        = train['time_id'].map(dict(zip(time['time_id'], time['std'])))

# train_info['row_id_mean']      = train['row_id'].map(dict(zip(row['row_id'], row['mean'])))
# train_info['row_id_median']    = train['row_id'].map(dict(zip(row['row_id'], row['median'])))
# train_info['row_id_std']        = train['row_id'].map(dict(zip(row['row_id'], row['std'])))


In [12]:
book_train = pd.read_parquet(data_dir + "book_train.parquet/stock_id=15")
book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,0.999519,0.999839,0.999454,0.999904,2,166,2,12
1,5,1,0.999711,1.000225,0.999647,1.000289,100,20,100,20
2,5,2,0.999775,1.000225,0.999711,1.000289,1,20,400,20
3,5,3,0.999839,1.000225,0.999775,1.000289,100,20,1,20
4,5,4,0.999839,1.000225,0.999711,1.000289,1,20,400,20


price_spread: 売値と買値の差のこと。スプレッドが広いということは、流動性が低いことを意味し、ボラティリティーが高いことにつながります。  

volume:  アスク/ビッドサイズの合計。出来高が少ないということは、流動性が低いということであり、ボラティリティーの高さにつながる    
 
volume_imbalance: アスクサイズとビッドサイズの差。インバランスが大きいと、片側の流動性が低く、ボラティリティーが高くなることを意味します。  

In [13]:
def preprocessor_book(file_path):
    df = pd.read_parquet(file_path)
    df["wap"] = calc_wap(df)

    # (stock_idはすべて同じ)ある時間のwapのlogの差
    df["log_return"] = df.groupby('time_id')["wap"].apply(log_return)

    df["wap2"] = calc_wap2(df)
    df["log_return2"] = df.groupby('time_id')["wap2"].apply(log_return)

    df["wap_balance"] = abs(df["wap"]-df["wap2"])

    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']

    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

    create_feature_dict = {
        'log_return':[realized_vocatility],
        'log_return2':[realized_vocatility],
        'wap_balance':[np.mean],
        'price_spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean],
        'wap':[np.mean]
    }

    df_feature = df.groupby(['time_id']).agg(create_feature_dict).reset_index()
    df_feature.columns = ['_'.join(col)for col in df_feature.columns]
    # print("##",df_feature.columns[0])

    ######groupby / last XX seconds
    last_seconds = [300]
    
    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        # print(df_feature.columns)
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    #create row_id
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)
    
    return df_feature

In [14]:
file_path = data_dir + "book_train.parquet/stock_id=0"
preprocessor_book(file_path)


,log_return_realized_vocatility,log_return2_realized_vocatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,log_return_realized_vocatility_300,log_return2_realized_vocatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,wap_mean_300,row_id
0,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,0.002953,0.004863,0.000372,0.000822,0.000223,-0.000162,137.158273,294.928058,1.003753,0-5
1,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,135.513043,484.521739,1.000397,0-11
2,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,144.147059,455.235294,0.998685,0-16
3,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,144.698113,418.169811,0.998436,0-31
4,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,99.449438,407.584270,0.999488,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.002579,0.003821,0.000212,0.000552,0.000083,-0.000182,197.144781,374.235690,0.997938,0.001673,0.002573,0.000193,0.000509,0.000062,-0.000169,233.946667,350.560000,0.997519,0-32751
3826,0.002206,0.002847,0.000267,0.000542,0.000092,-0.000172,233.781553,621.131068,1.000310,0.001487,0.002255,0.000300,0.000588,0.000074,-0.000177,257.920000,668.640000,1.000682,0-32753
3827,0.002913,0.003266,0.000237,0.000525,0.000202,-0.000083,115.829787,343.734043,0.999552,0.001929,0.002646,0.000216,0.000446,0.000191,-0.000075,105.432692,326.759615,1.000111,0-32758
3828,0.003046,0.005105,0.000245,0.000480,0.000113,-0.000166,132.074919,385.429967,1.002357,0.002137,0.003934,0.000269,0.000516,0.000096,-0.000175,123.423313,394.588957,1.002277,0-32763


In [15]:
trade_train = pd.read_parquet(data_dir + "trade_train.parquet/stock_id=0")
trade_train.head(15)

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1
5,5,78,1.003762,134,5
6,5,122,1.004207,102,3
7,5,127,1.004577,1,1
8,5,144,1.004370,6,1
9,5,147,1.003964,233,4


In [16]:
def preprocessor_trade(file_path):
    df = pd.read_parquet(file_path)
    df["log_return"] = df.groupby(["time_id"])["price"].apply(log_return)

    aggregate_dictionary = {
        'log_return':[realized_vocatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }

    df_feature = df.groupby(["time_id"]).agg(aggregate_dictionary).reset_index()
    df_feature.columns = ["_".join(col)for col in df_feature.columns]

    last_seconds = [300]
    for second in last_seconds:
        second = 600-second
        df_feature_sec = df.query(f'seconds_in_bucket>={second}').groupby(["time_id"]).agg(aggregate_dictionary).reset_index()
        df_feature_sec.columns = ["_".join(col) for col in df_feature_sec.columns]
        
        # サフィックス（接尾辞）に追加
        # 例　A -> A_X
        df_feature_sec = df_feature_sec.add_suffix("_"+str(second))
        df_feature = pd.merge(df_feature,df_feature_sec,how="left",left_on="time_id_",right_on=f'time_id__{second}')
        df_feature.drop([f'time_id__{second}'],axis=1,inplace=True)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature["row_id"] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(["trade_time_id_"],axis=1,inplace=True)

    return df_feature


In [17]:
file_path =  data_dir + "trade_train.parquet/stock_id=0"
preprocessor_trade(file_path)

,trade_log_return_realized_vocatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_vocatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,row_id
0,0.002006,40,3179,2.750000,0.001308,21.0,1587.0,2.571429,0-5
1,0.000901,30,1289,1.900000,0.000587,16.0,900.0,2.250000,0-11
2,0.001961,25,2161,2.720000,0.001137,12.0,1189.0,3.166667,0-16
3,0.001561,15,1962,3.933333,0.001089,9.0,1556.0,5.111111,0-31
4,0.000871,22,1791,4.045455,0.000453,11.0,1219.0,4.909091,0-62
...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,3450,3.057692,0.001162,35.0,2365.0,3.257143,0-32751
3826,0.001411,28,4547,3.892857,0.001066,12.0,2161.0,4.250000,0-32753
3827,0.001521,36,4250,3.500000,0.001242,22.0,2294.0,3.727273,0-32758
3828,0.001794,53,3217,2.150943,0.001404,25.0,1627.0,1.920000,0-32763


## Combined preprocessor function

In [18]:
def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(preprocessor_book(file_path_book),preprocessor_trade(file_path_trade),on='row_id',how='left')
     
        return pd.concat([df,df_tmp])
    
    # 並列処理してるらしい。今回はコピペ（詳しくはわからない）
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df


In [19]:
list_stock_ids = [0,1]
temp = preprocessor(list_stock_ids, is_train = True)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    8.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    8.4s finished


In [20]:
train = pd.read_csv(f'{data_dir}train.csv')
train_ids = train.stock_id.unique()

In [21]:
df_train = preprocessor(list_stock_ids=train_ids,is_train=True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
df_train

,log_return_realized_vocatility,log_return2_realized_vocatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,log_return_realized_vocatility_300,log_return2_realized_vocatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,wap_mean_300,row_id,trade_log_return_realized_vocatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_vocatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,0.002953,0.004863,0.000372,0.000822,0.000223,-0.000162,137.158273,294.928058,1.003753,0-5,0.002006,40.0,3179.0,2.750000,0.001308,21.0,1587.0,2.571429
1,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,135.513043,484.521739,1.000397,0-11,0.000901,30.0,1289.0,1.900000,0.000587,16.0,900.0,2.250000
2,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,144.147059,455.235294,0.998685,0-16,0.001961,25.0,2161.0,2.720000,0.001137,12.0,1189.0,3.166667
3,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,144.698113,418.169811,0.998436,0-31,0.001561,15.0,1962.0,3.933333,0.001089,9.0,1556.0,5.111111
4,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,99.449438,407.584270,0.999488,0-62,0.000871,22.0,1791.0,4.045455,0.000453,11.0,1219.0,4.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,0.003691,0.005876,0.000361,0.000878,0.000091,-0.000202,161.638710,406.045161,0.999582,0.002899,0.003776,0.000297,0.000890,0.000091,-0.000162,150.471831,379.443662,0.999375,126-32751,0.002171,37.0,2570.0,2.783784,0.001451,18.0,796.0,2.055556
428928,0.004104,0.004991,0.000295,0.000706,0.000126,-0.000142,150.578475,243.322870,1.002476,0.003454,0.003402,0.000280,0.000729,0.000147,-0.000168,153.819048,242.561905,1.003528,126-32753,0.002180,43.0,2323.0,3.418605,0.001791,20.0,1107.0,3.550000
428929,0.003118,0.006019,0.000394,0.000739,0.000189,-0.000192,254.406250,348.093750,1.001082,0.002792,0.005387,0.000430,0.000704,0.000244,-0.000200,249.401198,343.592814,1.001282,126-32758,0.001921,35.0,3740.0,2.800000,0.001580,24.0,2750.0,2.541667
428930,0.003661,0.005362,0.000231,0.000530,0.000143,-0.000134,145.654135,426.416040,1.001809,0.002379,0.003182,0.000199,0.000493,0.000150,-0.000118,159.832461,471.183246,1.001807,126-32763,0.002051,80.0,9389.0,2.925000,0.001520,43.0,5150.0,2.813953


In [ ]:
train["row_id"] = train["stock_id"].astype(str)+"-"+train["time_id"].astype(str)
train = train[["row_id","target"]]
df_train = train.merge(df_train,on="row_id",how="left")

In [ ]:
test = pd.read_csv(f'{data_dir}test.csv')
test_ids = test.stock_id.unique()

In [ ]:
df_test = preprocessor(list_stock_ids=test_ids,is_train=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.9s finished


## Target encoding by stock_id

In [ ]:
from sklearn.model_selection import KFold

df_train["stock_id"] = df_train["row_id"].apply(lambda x:x.split("-")[0])
df_test["stock_id"] = df_test["row_id"].apply(lambda x:x.split("-")[0])

In [ ]:
stock_id_target_mean = df_train.groupby("stock_id")["target"].mean()

In [ ]:
df_test["stock_id_target_enc"] = df_test["stock_id"].map(stock_id_target_mean)

In [ ]:
tmp = np.repeat(np.nan,df_train.shape[0]) #nanの配列
kf = KFold(n_splits=10,shuffle=True,random_state=123)
for idx_1,idx_2 in kf.split(df_train):
    # 検証用データ(idx_1)でtargetの平均をとりテスト用のidxに平均値を入れてる
    # stock_idごとに分けて平均とってるので更新される可能性があるのでは？
    # 更新されてるならその平均をとる必要がありそう。
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()
    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

In [ ]:
# len(df_train['stock_id_target_enc'].unique())

1120

# Model Building

In [ ]:
DO_FEAT_IMP = False
if len(df_test)==3:
    DO_FEAT_IMP = True

## LightGBM

In [ ]:
import lightgbm as lgbm

In [ ]:
# ref https://www.kaggle.com/corochann/permutation-importance-for-feature-selection-part1
def calc_model_importance(model, feature_names=None, importance_type='gain'):
    importance_df = pd.DataFrame(model.feature_importance(importance_type=importance_type),
                                 index=feature_names,
                                 columns=['importance']).sort_values('importance')
    return importance_df


def plot_importance(importance_df, title='',
                    save_filepath=None, figsize=(8, 12)):
    fig, ax = plt.subplots(figsize=figsize)
    importance_df.plot.barh(ax=ax)
    if title:
        plt.title(title)
    plt.tight_layout()
    if save_filepath is None:
        plt.show()
    else:
        plt.savefig(save_filepath)
    plt.close()
    

In [ ]:
df_train["stock_id"] = df_train["stock_id"].astype(int)
df_test["stock_id"] = df_test["stock_id"].astype(int)


In [ ]:
X = df_train.drop(["row_id","target"],axis=1)
y = df_train["target"]

In [ ]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_RMSPE(preds, lgbm_train):
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
  }

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,random_state=123,shuffle=True)
oof = pd.DataFrame()
models = []
scores = 0.0
gain_importance_list = []
split_importance_list = []


In [ ]:
for fold ,(train_idx,val_index) in enumerate(kf.split(X,y)):
    print(f'Fold : {fold+1}')
    X_train,y_train = X.loc[train_idx],y.loc[train_idx]
    X_valid,y_valid = X.loc[val_index],y.loc[val_index]

    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train,y_train,weight=weights)

    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid,y_valid,weight=weights)

    model = lgbm.train(
        params=params,
        train_set=lgbm_train,
        valid_sets=[lgbm_train,lgbm_valid],
        num_boost_round=5000,
        feval = feval_RMSPE,
        verbose_eval=100,
        categorical_feature=["stock_id"]    
    )

    y_pred = model.predict(X_valid,num_iteration=model.best_iteration)
    RMSPE = round(rmspe(y_valid,y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')
    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)
    
    # --- calc model feature importance ---
    if DO_FEAT_IMP:    
        feature_names = X_train.columns.values.tolist()
        gain_importance_df = calc_model_importance(
            model, feature_names=feature_names, importance_type='gain')
        gain_importance_list.append(gain_importance_df)

        split_importance_df = calc_model_importance(
            model, feature_names=feature_names, importance_type='split')
        split_importance_list.append(split_importance_df)